In [95]:
from shapely.geometry import box, Polygon, MultiPolygon, GeometryCollection

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

import psycopg2
import seaborn as sns
from sklearn.model_selection import train_test_split

from shapely.wkt import loads as wkt_loads
from matplotlib.collections import LineCollection
import matplotlib.cm as cm
import geopandas as gp

%matplotlib inline 

try:
    conn = psycopg2.connect("dbname='crops' user='postgres' host='localhost' port='5433'")
    cur = conn.cursor()
except:
    print "Unable to connect to the database"


In [66]:


soilDf = gp.read_postgis('select geom_fixed as geom, gronds, omschrijvi from "soil_map"', conn)
soilDf.head()        

,geom,gronds,omschrijvi
0,(POLYGON ((128228.0467999987 545672.5623999983...,40,Zware zavel
1,(POLYGON ((199736.1405999996 549441.0623999983...,21,Moerig op zand
2,(POLYGON ((183502.4217999987 549952.5623999983...,21,Moerig op zand
3,"(POLYGON ((205858.7655999996 549375.75, 205859...",70,Leem
4,"(POLYGON ((241182 474442, 241176 474397, 24116...",20,Zand


In [4]:

def katana(geometry, threshold, count=0):
    """Split a Polygon into two parts across it's shortest dimension"""
    bounds = geometry.bounds
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    if max(width, height) <= threshold or count == 250:
        # either the polygon is smaller than the threshold, or the maximum
        # number of recursions has been reached
        return [geometry]
    if height >= width:
        # split left to right
        a = box(bounds[0], bounds[1], bounds[2], bounds[1]+height/2)
        b = box(bounds[0], bounds[1]+height/2, bounds[2], bounds[3])
    else:
        # split top to bottom
        a = box(bounds[0], bounds[1], bounds[0]+width/2, bounds[3])
        b = box(bounds[0]+width/2, bounds[1], bounds[2], bounds[3])
    result = []
    for d in (a, b,):
        c = geometry.intersection(d)
        if not isinstance(c, GeometryCollection):
            c = [c]
        for e in c:
            if isinstance(e, (Polygon, MultiPolygon)):
                result.extend(katana(e, threshold, count+1))
    if count > 0:
        return result
    # convert multipart into singlepart
    final_result = []
    for g in result:
        if isinstance(g, MultiPolygon):
            final_result.extend(g)
        else:
            final_result.append(g)
    return final_result


In [31]:
testgeom = soilDf.ix[0,'geom']
splits = katana(testgeom,1000)


In [29]:
testgeom.length

6290.664962727452

In [32]:
splits[0].length

2832.989323593245

In [33]:
refDf = gp.read_postgis('select global_geo_id, geom, shape_area, centroid_x, centroid_y, gws_gewas as c2009 from "2009"', conn)


In [41]:
for i in xrange(5,15):
    print refDf.ix[i,'geom'].length

114.751613261
345.873706508
637.764742038
337.666032478
238.062555207
1869.83229449
681.973325332
326.592318946
168.348529986
1357.69427556


In [42]:
for i in xrange(5,15):
    print refDf.ix[i,'geom'].area

813.137205456
4404.98141104
22268.6136193
5507.79563551
868.225693094
10312.4408567
20920.564348
6437.08520496
579.325663402
48451.7520719


In [47]:
refDf['splits'] = refDf.geom.apply(lambda x : len(katana(x,1000)))

In [60]:
refDf['splits'].value_counts()

1     816098
2       2611
3        186
4        145
5         32
6         18
8         15
7          6
10         4
15         3
12         3
13         3
14         3
16         3
11         2
9          2
62         1
17         1
18         1
19         1
20         1
26         1
30         1
32         1
46         1
57         1
58         1
74         1
Name: splits, dtype: int64

1000 seems to be a good threshold value 

In [102]:
%time

for index, parent in soilDf.iterrows():
    splits = katana(parent.geom,1000)
    
    for child in splits:
        
        cur.execute('INSERT INTO soil_map_dump (omschrijvi, gronds, geom) VALUES (\''+\
                    str(parent.omschrijvi)+\
                    '\','+str(parent.gronds)+','+ 'ST_GeomFromText(\''+child.wkt+'\', 28992))')
                    
                    
#         cur.execute('INSERT INTO soil_map_dump (omschrijvi, gronds, geom) VALUES (%s, %s, %s)', \
#                     ( parent.omschrijvi, parent.gronds, child))
# #                      "ST_GeomFromText("+child.wkt+", 28992)"))

#         print mini_parcel
        
    

conn.commit()    
    

CPU times: user 4 µs, sys: 1e+03 ns, total: 5 µs
Wall time: 36 µs
